# Import Library

In [ ]:
import os
import requests
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from openai import OpenAI
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, losses, InputExample
from sentence_transformers.evaluation import InformationRetrievalEvaluator
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
### 구현 예시 (더 상세한 코드)
from sentence_transformers import SentenceTransformer, losses, InputExample
from torch.utils.data import DataLoader
import torch

# 모델 로드
model = SentenceTransformer('distilbert-base-multilingual-cased')

# 훈련 데이터 준비
train_examples = [
    InputExample(texts=["AI란 무엇인가?", "AI는 인간의 지능을 모방한 기술입니다."]),
    InputExample(texts=["딥러닝이란?", "신경망을 여러 층 쌓아 데이터로부터 학습하는 기계학습 방법입니다."]),
    InputExample(texts=["Python은 어디에 쓰이나요?", "Python은 데이터 분석, 웹 개발, AI 등에 널리 사용됩니다."]),
    InputExample(texts=["자연어 처리란?", "컴퓨터가 인간의 언어를 이해하고 처리하는 AI의 한 분야입니다."])
]

# 배치 크기가 클수록 성능이 향상될 수 있음
batch_size = 32  # 실제 환경에서는 16-64 범위가 일반적
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

# MultipleNegativesRankingLoss 설정
# 온도(temperature) 파라미터를 조정하여 손실 함수의 강도 조절 가능
loss = losses.MultipleNegativesRankingLoss(model, scale=20.0)  # scale은 temperature의 역수

# 학습 설정
train_loss = losses.MultipleNegativesRankingLoss(model)
warmup_steps = int(len(train_dataloader) * 0.1)  # 전체 훈련 데이터의 10%

# 모델 학습
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=warmup_steps,
    optimizer_params={'lr': 2e-5},
    output_path='./korean-sentence-embedding-model'
)
